## ResNet-50 Model:
---
Developed by: Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun

Published in: 2015

Primary Use Case: Image classification

---

### Architecture:
Input Layer: 224x224 RGB images

Initial Convolution and Pooling:

7x7 convolution with 64 filters, stride 2

3x3 max pooling with stride 2

Residual Blocks:

Layer 1: 3 Basic Blocks with 64 filters

Layer 2: 4 Basic Blocks with 128 filters

Layer 3: 6 Basic Blocks with 256 filters

Layer 4: 3 Basic Blocks with 512 filters

Global Average Pooling: Converts feature maps into a 1D feature vector

Fully Connected Layer: Number of classes (usually 1000 for ImageNet)

Activation Functions: Uses ReLU activation function after each convolutional layer

Batch Normalization: Applied after each convolutional layer

---
### Key Features:
Residual Learning: Introduces skip connections to bypass one or more layers, addressing the vanishing gradient problem and enabling the training of very deep networks.

Deep Architecture: Contains 50 layers, making it one of the deepest networks at the time of its release.

Performance: Achieved state-of-the-art results on the ImageNet dataset, demonstrating the effectiveness of deep residual learning.

---
### Applications:
Image Classification: Used for classifying images into different categories.

Feature Extraction: Pre-trained ResNet-50 models are often used to extract features from images for transfer learning in other tasks.

###
### Impact:
Influence: ResNet-50 has influenced the design of many subsequent deep learning models with its innovative residual learning approach.

Benchmark: Often used as a benchmark for comparing the performance of new models and techniques in the field of computer vision.

---

---

In [3]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [5]:
# Create an instance of the ResNet-50 model
model = ResNet(num_classes=1000)

In [7]:

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  